In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Get started with XGBoost on Ray on Vertex AI
{TODO: Update the links below.}

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/notebook_template.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/notebook_template.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/notebook_template.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>                                                                                               
</table>

**_NOTE_**: This notebook has been tested in the following environment:

* Python version = 3.9

## Overview



Learn more about [web-doc-title](linkback-to-webdoc-page). {TODO: if more than one primary feature, add tag/linkback for each one}

### Objective

In this notebook, you learn how to preprocess, train and serve a XGBoost model using Ray on Vertex AI.

This tutorial uses the following Google Cloud ML services and resources:

- Cloud storage
- BigQuery
- Ray on Vertex AI
- Vertex AI Prediction

The steps performed include:

- Data preprocessing
- Fine tuning
- Model serving for online inference

### Dataset

The BigQuery `ulb_fraud_detection` dataset contains transactions made by credit cards in September 2013 by european cardholders. This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284,807 transactions. It is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.



### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage


Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing),
and [Cloud Storage pricing](https://cloud.google.com/storage/pricing),
and use the [Pricing Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Installation

Install the following packages required to execute this notebook.

In [ ]:
# Install the packages
! pip3 install --user --upgrade pip --force-reinstall --no-warn-conflicts -q
! pip3 install --user --upgrade gputil etils --force-reinstall --no-warn-conflicts -q
! pip3 install --user --upgrade 'google-cloud-aiplatform[ray]==1.33.1' 'ray[air]==2.4' 'xgboost==1.7.6' 'xgboost_ray==0.1.18' --force-reinstall --no-warn-conflicts -q

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.4/173.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 5.0 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directo

### Colab only: Uncomment the following cell to restart the kernel.

In [ ]:
# # Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## Before you begin

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

3. [Enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com). {TODO: Update the APIs needed for your tutorial. Edit the API names, and update the link to append the API IDs, separating each one with a comma. For example, container.googleapis.com,cloudbuild.googleapis.com}

4. If you are running this notebook locally, you need to install the [Cloud SDK](https://cloud.google.com/sdk).

#### Set your project ID

**If you don't know your project ID**, try the following:
* Run `gcloud config list`.
* Run `gcloud projects list`.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

PROJECT_ID = "gen-ai-workshop"  # @param {type:"string"}


# Set the project id
! gcloud config set project {PROJECT_ID}

Updated property [core/project].


#### Region

You can also change the `REGION` variable used by Vertex AI. Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### Timestamp

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d-%H%M%S")

### Authenticate your Google Cloud account

Depending on your Jupyter environment, you may have to manually authenticate. Follow the relevant instructions below.

**1. Vertex AI Workbench**
* Do nothing as you are already authenticated.

**2. Local JupyterLab instance, uncomment and run:**

In [ ]:
# ! gcloud auth login

**3. Colab, uncomment and run:**

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

**4. Service account or other**
* See how to grant Cloud Storage permissions to your service account at https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples.

### Create a Cloud Storage bucket

Create a storage bucket to store intermediate artifacts such as datasets.

- *{Note to notebook author: For any user-provided strings that need to be unique (like bucket names or model ID's), append "-unique" to the end so proper testing can occur}*

In [ ]:
# BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

BUCKET_URI = f"gs://ray-on-vertex-{TIMESTAMP}"  # @param {type:"string"}

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}

Creating gs://ray-on-vertex-20231010-044524/...


### Project number

In [ ]:
PROJECT_NUMBER = !gcloud projects list --filter="$(gcloud config get-value project)" --format="value(PROJECT_NUMBER)"
PROJECT_NUMBER = PROJECT_NUMBER[0]

### Prepare BigQuery dataset


In [ ]:
from google.cloud import bigquery

In [ ]:
LOCATION = REGION.split('-')[0]
DATASET_ID = 'rov_xgboost_dataset'
TABLE_ID = f'fraud_detection_{TIMESTAMP}'

In [ ]:
def create_bq_dataset(dataset_id):
  bq_client = bigquery.Client(project=PROJECT_ID, location=LOCATION)
  dataset_uri = f'{bq_client.project}.{dataset_id}'
  dataset = bigquery.Dataset(dataset_uri)
  dataset = bq_client.create_dataset(dataset, exists_ok=True)

def run_bq_query(sql):
  bq_client = bigquery.Client(project=PROJECT_ID, location=LOCATION)
  job_config = bigquery.QueryJobConfig()
  job = bq_client.query(sql, job_config=job_config)
  job.result()

In [ ]:
_ = create_bq_dataset(DATASET_ID)

In [ ]:
sql = f'CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}` AS SELECT * FROM `bigquery-public-data.ml_datasets.ulb_fraud_detection`'
_ = run_bq_query(sql)

**⚠️ BE MINDFUL TO RUN.**

The query below will create a +150 GB on BigQuery. It is for stress test only.

In [ ]:
sql = f'''
DECLARE counter INT64;
SET counter = 1;

# Loop the query X times.
WHILE counter <= 10 DO

  # Append the data from the original table to the new table.
  INSERT INTO `{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}`
  SELECT *
  FROM `{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}`;

  # Increment the counter variable.
  SET counter = counter + 1;

END WHILE;
'''
_ = run_bq_query(sql)

NameError: ignored

### Set tutorial folder

In [ ]:
from pathlib import Path as path

root_path = path.cwd()
tutorial_path = root_path / 'xgb_rov_tutorial'
tutorial_path.mkdir(parents=True, exist_ok=True)

## Preprocess, train and serve an XGBoost using Ray on Vertex AI

### Import libraries

In [ ]:
# General
import uuid
import random
import time
import GPUtil as gputil
from etils import epath
from pprint import pprint

# Ray
import ray
from ray.runtime_env import RuntimeEnv
from ray.air.config import RunConfig
from ray.air import CheckpointConfig, ScalingConfig
from ray.tune.syncer import SyncConfig
from ray.train.xgboost import XGBoostTrainer
from ray.train.xgboost import XGBoostCheckpoint

# Ray on Vertex AI
from google.cloud import aiplatform as vertex_ai
import vertex_ray
from vertex_ray import BigQueryDatasource
from vertex_ray.predict import xgboost

In [ ]:
print("Ray version:", ray.__version__)

Ray version: 2.4.0


### Define constants

In [ ]:
# Data
PARALLELISM = 8
QUERY = f'SELECT * EXCEPT(Time) FROM `{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}`'

# Train
TRAIN_FOLDER_URI = str(epath.Path(BUCKET_URI) / 'logs')

# Serve
ENDPOINT_NAME = f"rov-xgboost-endpoint-{TIMESTAMP}"
DEPLOYED_MODEL_NAME = f"rov-xgboost-model-{TIMESTAMP}"
TRAFFIC_SPLIT = {"0": 100}
MACHINE_TYPE = "n1-standard-4"
MIN_NODES = 1
MAX_NODES = 1

### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project.

In [ ]:
vertex_ai.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

### Initialize Ray cluster on Vertex AI

Initialize the Vertex AI SDK for Python for your project.

In [ ]:
ray_clusters = vertex_ray.list_ray_clusters()
ray_cluster_resource_name = ray_clusters[0].cluster_resource_name
ray_cluster = vertex_ray.get_ray_cluster(ray_cluster_resource_name)

In [ ]:
requirements = """
google-cloud-aiplatform[ray]==1.33.1
ray[air]==2.4
xgboost==1.7.6
xgboost_ray==0.1.18
gcsfs
etils
ipython
importlib_resources
"""

with open(tutorial_path / "requirements.txt", "w") as f:
    f.write(requirements)
f.close()

In [ ]:
runtime_env = RuntimeEnv(
    pip={"packages": str(tutorial_path / "requirements.txt")}
)

In [ ]:
ray.shutdown()
ray_cluster_address = f'vertex_ray://{ray_cluster_resource_name}'
ray.init(runtime_env=runtime_env, address=ray_cluster_address)

[Ray on Vertex AI]: Using cluster resource name to access head address with GAPIC API


INFO:google.cloud.aiplatform.preview.vertex_ray.client_builder:[Ray on Vertex AI]: Using cluster resource name to access head address with GAPIC API


[Ray on Vertex AI]: Cluster State = State.RUNNING
[Ray on Vertex AI]: Connecting...


INFO:google.cloud.aiplatform.preview.vertex_ray.client_builder:[Ray on Vertex AI]: Connecting...


Python version:,3.10.12
Ray version:,2.4.0
Vertex SDK version:,1.33.1
Dashboard:,c6e02b109cff463f-dot-us-central1.aiplatform-training.googleusercontent.com
Interactive Terminal Uri:,0b79269e4838fec9-dot-us-central1.aiplatform-training.googleusercontent.com
Cluster Name:,ray-cluster-20231008-145921


In [ ]:
pprint(ray.cluster_resources())

{'CPU': 15.0,
 'memory': 60848505242.0,
 'node:10.15.0.3': 1.0,
 'node:10.15.0.4': 1.0,
 'object_store_memory': 28079810150.0}


### Distributed data ingestion from BigQuery

In [ ]:
# with parallelism
bq_dataset = ray.data.read_datasource(
    BigQueryDatasource(),
    query=QUERY,
    parallelism=PARALLELISM,
)

2023-10-10 07:22:25,750	WARNING read_api.py:358 -- ⚠️  The number of blocks in this dataset (2) limits its parallelism to 2 concurrent tasks. This is much less than the number of available CPU slots in the cluster. Use `.repartition(n)` to increase the number of dataset blocks.


In [ ]:
# preprocess
bq_train_dataset, bq_valid_dataset = bq_dataset.train_test_split(test_size=0.3, seed=8)

Read progress 0:   0%|          | 0/2 [00:00<?, ?it/s]

(_get_read_tasks pid=22391) [Ray on Vertex AI]: Created streams: 2
(_get_read_tasks pid=22391) [Ray on Vertex AI]: The number of streams created by the BigQuery Storage Read API is less than the requested parallelism due to the size of the dataset.


Read progress 0:   0%|          | 0/2 [00:00<?, ?it/s]

###  Distributed training with XGBoost model

In [ ]:
id = "".join(random.sample(str(uuid.uuid4()), 5))

# xgboost configuration
xgboost_config = {
  "objective": "binary:logistic",
  "eval_metric": ["logloss", "error"],
  "num_boost_round": 20,
}
xgboost_config["tree_method"] = "gpu_hist" if 'GPU' in ray.cluster_resources().keys() else "hist"

# scaling config
scaling_config = ScalingConfig(
  num_workers=2,
  use_gpu=True if 'GPU' in ray.cluster_resources().keys() else False,
)

# run config
run_config = RunConfig(
  checkpoint_config=CheckpointConfig(
    num_to_keep=None
  ),
  sync_config=SyncConfig(
      upload_dir=TRAIN_FOLDER_URI
  ),
  name=f"train-xgb-on-rov-{id}",
)

# train
trainer = XGBoostTrainer(
  scaling_config=scaling_config,
  run_config=run_config,
  label_column="Class",
  num_boost_round=20,
  params=xgboost_config,
  datasets={"train": bq_train_dataset, "valid": bq_valid_dataset},
)

In [ ]:
result = trainer.fit()

(XGBoostTrainer pid=22666) 2023-10-10 07:22:40,342	INFO streaming_executor.py:83 -- Executing DAG InputDataBuffer[Input] -> AllToAllOperator[Repartition]
(XGBoostTrainer pid=22666) 2023-10-10 07:22:40,342	INFO streaming_executor.py:84 -- Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=None), locality_with_output=False, preserve_order=False, actor_locality_enabled=True, verbose_progress=False)
(XGBoostTrainer pid=22666) 2023-10-10 07:22:43,254	INFO tracker.py:218 -- start listen on 10.15.0.4:35617
(_RemoteRayXGBoostActor pid=7000, ip=10.15.0.3) [07:22:43] task [xgboost.ray]:138972929250192 got new rank 0
(_RemoteRayXGBoostActor pid=7001, ip=10.15.0.3) [07:22:43] task [xgboost.ray]:133763604780064 got new rank 1
(XGBoostTrainer pid=22666) 2023-10-10 07:22:43,528	INFO tracker.py:382 -- @tracker All of 2 nodes getting started
(_RemoteRayXGBoostActor pid=7000, ip=10.15.0.3) /tmp/ray/session_2023-10-08_15-09-03_341832_8/runtime_re

(TunerInternal pid=22542) Result for XGBoostTrainer_c77ff_00000:
(TunerInternal pid=22542)   date: 2023-10-10_07-22-45
(TunerInternal pid=22542)   done: false
(TunerInternal pid=22542)   hostname: gke-vertex-persistent-1008--head-node-9e64c6e4-m2gm
(TunerInternal pid=22542)   iterations_since_restore: 1
(TunerInternal pid=22542)   node_ip: 10.15.0.4
(TunerInternal pid=22542)   pid: 22666
(TunerInternal pid=22542)   time_since_restore: 7.933728456497192
(TunerInternal pid=22542)   time_this_iter_s: 7.933728456497192
(TunerInternal pid=22542)   time_total_s: 7.933728456497192
(TunerInternal pid=22542)   timestamp: 1696922565
(TunerInternal pid=22542)   train-error: 0.0
(TunerInternal pid=22542)   train-logloss: 0.4374922215938568
(TunerInternal pid=22542)   training_iteration: 1
(TunerInternal pid=22542)   trial_id: c77ff_00000
(TunerInternal pid=22542)   valid-error: 0.00574658832892489
(TunerInternal pid=22542)   valid-logloss: 0.44094010502471814
(TunerInternal pid=22542)   


(XGBoostTrainer pid=22666) 2023-10-10 07:22:45,835	INFO tracker.py:388 -- @tracker All nodes finishes job


(TunerInternal pid=22542) Result for XGBoostTrainer_c77ff_00000:
(TunerInternal pid=22542)   date: 2023-10-10_07-22-46
(TunerInternal pid=22542)   done: true
(TunerInternal pid=22542)   experiment_tag: '0'
(TunerInternal pid=22542)   hostname: gke-vertex-persistent-1008--head-node-9e64c6e4-m2gm
(TunerInternal pid=22542)   iterations_since_restore: 21
(TunerInternal pid=22542)   node_ip: 10.15.0.4
(TunerInternal pid=22542)   pid: 22666
(TunerInternal pid=22542)   should_checkpoint: true
(TunerInternal pid=22542)   time_since_restore: 8.924169063568115
(TunerInternal pid=22542)   time_this_iter_s: 0.24509096145629883
(TunerInternal pid=22542)   time_total_s: 8.924169063568115
(TunerInternal pid=22542)   timestamp: 1696922566
(TunerInternal pid=22542)   train-error: 0.0
(TunerInternal pid=22542)   train-logloss: 0.00110950006637722
(TunerInternal pid=22542)   training_iteration: 21
(TunerInternal pid=22542)   trial_id: c77ff_00000
(TunerInternal pid=22542)   valid-error: 0.005746588328924

(TunerInternal pid=22542) 2023-10-10 07:22:48,399	WARNING experiment_state.py:279 -- Experiment checkpoint syncing has been triggered multiple times in the last 30.0 seconds. A sync will be triggered whenever a trial has checkpointed more than `num_to_keep` times since last sync or if 300 seconds have passed since last sync. If you have set `num_to_keep` in your `CheckpointConfig`, consider increasing the checkpoint frequency or keeping more checkpoints. You can supress this warning by changing the `TUNE_WARN_EXCESSIVE_EXPERIMENT_CHECKPOINT_SYNC_THRESHOLD_S` environment variable.
(TunerInternal pid=22542) 2023-10-10 07:22:48,405	INFO tune.py:945 -- Total run time: 15.13 seconds (13.25 seconds for the tuning loop).


In [ ]:
ray.shutdown()

### Deploy a XGBoost model on Vertex AI

#### Register the model in Vertex AI Model Registry

In [ ]:
model_file_uri = result.checkpoint

In [ ]:
model = xgboost.register_xgboost(
    XGBoostCheckpoint.from_checkpoint(result.checkpoint),
)

Creating Model


INFO:google.cloud.aiplatform.models:Creating Model


Create Model backing LRO: projects/661110192441/locations/us-central1/models/4149598664668479488/operations/7960164774338101248


INFO:google.cloud.aiplatform.models:Create Model backing LRO: projects/661110192441/locations/us-central1/models/4149598664668479488/operations/7960164774338101248


Model created. Resource name: projects/661110192441/locations/us-central1/models/4149598664668479488@1


INFO:google.cloud.aiplatform.models:Model created. Resource name: projects/661110192441/locations/us-central1/models/4149598664668479488@1


To use this Model in another session:


INFO:google.cloud.aiplatform.models:To use this Model in another session:


model = aiplatform.Model('projects/661110192441/locations/us-central1/models/4149598664668479488@1')


INFO:google.cloud.aiplatform.models:model = aiplatform.Model('projects/661110192441/locations/us-central1/models/4149598664668479488@1')


#### Create a Vertex AI Endpoint

In [ ]:
endpoint = vertex_ai.Endpoint.create(
    display_name=ENDPOINT_NAME,
    description="A simple endpoint for xgboost trained on Ray on Vertex AI",
    sync=True,
)

Creating Endpoint


INFO:google.cloud.aiplatform.models:Creating Endpoint


Create Endpoint backing LRO: projects/661110192441/locations/us-central1/endpoints/736499067772731392/operations/6216145818638876672


INFO:google.cloud.aiplatform.models:Create Endpoint backing LRO: projects/661110192441/locations/us-central1/endpoints/736499067772731392/operations/6216145818638876672


Endpoint created. Resource name: projects/661110192441/locations/us-central1/endpoints/736499067772731392


INFO:google.cloud.aiplatform.models:Endpoint created. Resource name: projects/661110192441/locations/us-central1/endpoints/736499067772731392


To use this Endpoint in another session:


INFO:google.cloud.aiplatform.models:To use this Endpoint in another session:


endpoint = aiplatform.Endpoint('projects/661110192441/locations/us-central1/endpoints/736499067772731392')


INFO:google.cloud.aiplatform.models:endpoint = aiplatform.Endpoint('projects/661110192441/locations/us-central1/endpoints/736499067772731392')


#### Deploy the model to Vertex AI Endpoint

In [ ]:
deployed_model = endpoint.deploy(
    model=model,
    deployed_model_display_name=DEPLOYED_MODEL_NAME,
    traffic_split=TRAFFIC_SPLIT,
    machine_type=MACHINE_TYPE,
    min_replica_count=MIN_NODES,
    max_replica_count=MAX_NODES,
    sync = True,
)

Deploying Model projects/661110192441/locations/us-central1/models/4149598664668479488 to Endpoint : projects/661110192441/locations/us-central1/endpoints/736499067772731392


INFO:google.cloud.aiplatform.models:Deploying Model projects/661110192441/locations/us-central1/models/4149598664668479488 to Endpoint : projects/661110192441/locations/us-central1/endpoints/736499067772731392


Deploy Endpoint model backing LRO: projects/661110192441/locations/us-central1/endpoints/736499067772731392/operations/7083088746907697152


INFO:google.cloud.aiplatform.models:Deploy Endpoint model backing LRO: projects/661110192441/locations/us-central1/endpoints/736499067772731392/operations/7083088746907697152


Endpoint model deployed. Resource name: projects/661110192441/locations/us-central1/endpoints/736499067772731392


INFO:google.cloud.aiplatform.models:Endpoint model deployed. Resource name: projects/661110192441/locations/us-central1/endpoints/736499067772731392


### Generate predictions

In [ ]:
instances = [
  [-4.516, 1.171, 0.011, 1.063, 0.721, 1.094, -0.706, -4.826, -0.084, 1.583, 1.832, 0.539, -0.998, 0.516, 1.167, -0.967, 0.406, -0.568, 0.407, -0.638, 0.22, 0.386, -0.312, -0.271, 0.215, -0.314, -2.576, 0.565, 17.1],
  [1.906, -0.503, -1.351, 0.148, 1.726, 4.076, -1.321, 1.126, 0.818, 0.23, -0.035, 0.297, 0.131, 0.137, 1.652, 0.745, -1.069, 0.293, -0.994, -0.131, 0.122, 0.297, 0.311, 0.653, -0.341, -0.735, 0.09, -0.023, 17.1],
  [-3.3, -1.245, 1.475, 0.545, 1.398, -0.048, -0.321, 0.551, 0.091, -0.427, 0.947, 0.876, -0.858, 0.1, -1.162, -0.396, 0.051, -0.703, -1.229, -0.449, 0.064, 0.244, -0.685, -0.245, 0.52, -0.443, -0.72, 0.133, 17.1]
]

In [ ]:
predictions = endpoint.predict(instances=instances).predictions
print(predictions)

[0.00110890029463917, 0.00110890029463917, 0.00110890029463917]


## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial.

In [ ]:
import os

delete_bucket = True
delete_bigquery_dataset = True
delete_endpoint = True
delete_model = True


if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r {BUCKET_URI}

if delete_bigquery_dataset or os.getenv("IS_TESTING"):
    ! bq rm -f {PROJECT_ID}.{DATASET_ID}

if delete_endpoint or os.getenv("IS_TESTING"):
    endpoint.delete(force=True)

if delete_model or os.getenv("IS_TESTING"):
    model.delete()

BucketNotFoundException: 404 gs://ray-on-vertex-20231010-043918 bucket does not exist.
CommandException: Encountered non-existent bucket during listing
